In [6]:
#TODO funkcja trenująca powinna aktualizować słowniki w checkpoint.pth przypadku wczytania modelu
from data_helper import get_dataloaders_and_standarscaler_photons_from_numpy
from train_helper import train_vae
from plot_helper import plot_training_loss

import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn

In [7]:
# Hyperparameters
RANDOM_SEED = 123
NUM_EPOCHS = 1
LOGGING_INTERVAL=300

# RECONSTRUCTION_TERM_WEIGHT=2.8 
# LEARNING_RATE = 0.0005
# BATCH_SIZE = 1024
#TOTAL_NUM_EPOCHS
# SAVE_MODEL_FILE

PLOT_FRACTION=0.0125
TEST_FRACTION=0.4
VALIDATION_FRACTION=0.0
NUM_WORKERS=0
path='/data1/dose-3d-generative/data/training-data/DISP_0.5_ANGLE_0/NUMPY/a1_10_7.npy'
CHECKPOINT_PATH="TrainedModels/InfoVAE/30epoch_0203_mmd_checkpoint.pth"

In [8]:
loaded_checkpoint = torch.load(CHECKPOINT_PATH)

BATCH_SIZE = loaded_checkpoint["batch_size"]
RECONSTRUCTION_TERM_WEIGHT = loaded_checkpoint["reconstruction_term_weight"]
log_dict_old=loaded_checkpoint["log_dict"]

epoch = loaded_checkpoint["epoch"]
print(epoch)
TOTAL_NUM_EPOCHS=epoch+NUM_EPOCHS
model_name=loaded_checkpoint["model_name"]
print(model_name)

30
VAE_Linear_0203


In [9]:
SAVE_MODEL_FILE=f'TrainedModels/BetaVAE/{TOTAL_NUM_EPOCHS}epoch_0203_mmd_checkpoint.pth'#None gdy nie tworzy się nowego zapisu

In [10]:
CUDA_DEVICE_NUM=0
DEVICE = torch.device(f'cuda:{CUDA_DEVICE_NUM}' if torch.cuda.is_available() else 'cpu')
print('Device:', DEVICE)

Device: cuda:0


In [11]:
print(torch.cuda.memory_allocated(device=DEVICE))
print(torch.cuda.memory_reserved(device=DEVICE))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

39936
2097152
Quadro RTX 8000
50959679488


In [12]:
model = getattr(__import__('models_architecture_helper', fromlist=[model_name]), model_name)

In [13]:
model=model()
model.to(DEVICE)
model.load_state_dict(loaded_checkpoint["model_state"])

optimizer = torch.optim.Adam(model.parameters(),
                             lr=0, 
                             weight_decay=1e-5)

optimizer.load_state_dict(loaded_checkpoint["optim_state"])

#print(optimizer.state_dict)

In [14]:
#ODCZYTANIE DANYCH Z PLIKU 'photons.npy'
photons = np.load(path)
X = np.zeros((10000001, 6),dtype=np.float32)
np.copyto(X,photons[:,:-1])
print(X.shape)

# photons = np.load(path)
# X = np.zeros(photons.shape,dtype=np.float32)
# np.copyto(X,photons)

(10000001, 6)


In [15]:
X_reflection=copy.deepcopy(X)
X_reflection[:,2]=-X_reflection[:,2]
X_reflection[:,4]=-X_reflection[:,4]

X_sum=np.concatenate((X,X_reflection),axis=0)
print(len(X_sum))

20000002


In [16]:
df_data = pd.DataFrame(X_sum, columns = ['X', 'Y', 'dX', 'dY', 'dZ', 'E'])
df_data.head()#zawsze warto rzucić okiem na dane

,X,Y,dX,dY,dZ,E
0,0.77526,-0.50856,-4.60704,-0.018979,-0.177872,0.983871
1,0.77526,-0.77813,-5.10840,-0.029292,-0.196501,0.980066
2,0.45010,-3.74409,-1.33591,-0.145330,-0.051869,0.988023
3,0.39630,-2.50916,-3.30103,-0.274393,-0.273584,0.921879
4,0.45010,-4.79711,-1.70242,-0.184824,-0.065606,0.980579


In [17]:
train_loader, valid_loader, test_loader, stdcs = get_dataloaders_and_standarscaler_photons_from_numpy(tmp_X=X_sum,
    batch_size=BATCH_SIZE, 
    num_workers=NUM_WORKERS,
    test_fraction=TEST_FRACTION, 
    validation_fraction=VALIDATION_FRACTION)

KeyboardInterrupt: 

In [ ]:
# model = VAE_Linear()
# model.to(DEVICE)

#criterion = nn.MSELoss()#FUNKCJA STRATY
# optimizer = torch.optim.Adam(model.parameters(),
#                              lr=LEARNING_RATE, 
#                              weight_decay=1e-5)

In [ ]:
log_dict_new=train_vae(num_epochs=NUM_EPOCHS, device=DEVICE, model=model,optimizer=optimizer,train_loader=train_loader,loss_fn=None, test_loader=test_loader, logging_interval=LOGGING_INTERVAL, reconstruction_term_weight=RECONSTRUCTION_TERM_WEIGHT, save_model_file=SAVE_MODEL_FILE, total_num_of_epochs=TOTAL_NUM_EPOCHS)



In [ ]:
log_dict={ key:log_dict_old.get(key,[])+log_dict_new.get(key,[]) for key in set(list(log_dict_old.keys())+list(log_dict_new.keys())) }

In [ ]:
print(len(log_dict["test_combined_loss_per_epoch"]))

In [ ]:
epoch_all=epoch+NUM_EPOCHS
plot_training_loss(log_dict['train_reconstruction_loss_per_batch'], epoch_all, custom_label=" (reconstruction)")
plot_training_loss(log_dict['train_kl_loss_per_batch'], epoch_all, custom_label=" (KL)")
plot_training_loss(log_dict['train_combined_loss_per_batch'], epoch_all, custom_label=" (combined)")
plt.show()



In [ ]:
plt.plot(range(len(log_dict['train_combined_loss_per_epoch'])), (log_dict['train_combined_loss_per_epoch']), label='Train Epoch Loss')
plt.plot(range(len(log_dict['test_combined_loss_per_epoch'])), (log_dict['test_combined_loss_per_epoch']), label='Test Epoch Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
#plt.ylim(0.15,0.3)
plt.tight_layout()

In [ ]:
tmp=df_data.to_numpy(dtype=np.float32)
orginal=copy.deepcopy(tmp)
tmp=stdcs.transform(tmp)
tmp=torch.from_numpy(tmp)
#print(tmp)
model.eval()
with torch.no_grad():
    result_encoded_features, z_mean, z_log_var, result_decoded_features =model(tmp.to(device=DEVICE))
result=result_decoded_features.cpu().detach().numpy()
result=stdcs.inverse_transform(result)

In [ ]:
fig, axs = plt.subplots(2, 3)
fig.set_size_inches(16,8)
bins=100
axs[0, 0].hist(orginal[:,0],bins=bins, label ='orginal',alpha=0.5, density=True)
axs[0, 0].hist(result[:,0],bins=bins, label ='decoded', alpha=0.5, density=True)
axs[0, 0].set_title('X')
axs[0, 1].hist(orginal[:,1],bins=bins, label ='orginal',alpha=0.5, density=True)
axs[0, 1].hist(result[:,1],bins=bins, label ='decoded', alpha=0.5, density=True)
axs[0, 1].set_title('Y')
axs[0, 2].hist(orginal[:,2],bins=bins, label ='orginal',alpha=0.5, density=True)
axs[0, 2].hist(result[:,2],bins=bins, label ='decoded', alpha=0.5, density=True)
axs[0, 2].set_title('dX')
axs[1, 0].hist(orginal[:,3],bins=bins, label ='orginal',alpha=0.5, density=True)
axs[1, 0].hist(result[:,3],bins=bins, label ='decoded', alpha=0.5, density=True)
axs[1, 0].set_title('dY')
axs[1, 1].hist(orginal[:,4],bins=bins, label ='orginal',alpha=0.5, density=True)
axs[1, 1].hist(result[:,4],bins=bins, label ='decoded', alpha=0.5, density=True)
axs[1, 1].set_title('dZ')
axs[1, 2].hist(orginal[:,5],bins=bins, label ='orginal',alpha=0.5, density=True)
axs[1, 2].hist(result[:,5],bins=bins, label ='decoded', alpha=0.5, density=True)
axs[1, 2].set_title('E')
fig.legend()

# for ax in axs.flat:
#     ax.set(xlabel='x-label', ylabel='y-label')

# # Hide x labels and tick labels for top plots and y ticks for right plots.
# for ax in axs.flat:
#     ax.label_outer()
fig.tight_layout()

In [ ]:
keys = df_data.columns
# print(keys)
fig, axs = plt.subplots(2, 3)
fig.set_size_inches(20, 20)
for i, j in enumerate(keys):
    mi = np.minimum(orginal[:, i].min(), result[:, i].min())
    ma = np.maximum(orginal[:, i].max(), result[:, i].max())
    bins = np.linspace(mi, ma, 300)
    # print(bins)
    axs.flatten()[i].hist(orginal[:, i], bins, alpha=.5)
    axs.flatten()[i].hist(result[:, i], bins, alpha=.5)
    axs.flatten()[i].set_title(j)


In [ ]:
decoded_df=pd.DataFrame(result,columns=['X', 'Y', 'dX', 'dY', 'dZ', 'E'])
orginal_df=df_data.iloc[:,:]


concatenated_datasets=pd.concat([orginal_df.assign(dataset_name='orginal'), decoded_df.assign(dataset_name='decoded')],ignore_index=True)
concatenated_datasets.shape
print(concatenated_datasets.index.is_unique)

In [ ]:
#TODO trzeba by sprawdzić czy to losowanie działa poprawnie i zwraca próbkę reprezentatywną
sample_concatenated=concatenated_datasets.groupby('dataset_name', group_keys=False).apply(lambda x: x.sample(frac=PLOT_FRACTION, random_state=2)) 
print(sample_concatenated.shape)
print(sample_concatenated.head())

In [ ]:
sns.pairplot(sample_concatenated, hue="dataset_name", kind="scatter", plot_kws=dict(alpha=0.5))
#sns.pairplot(concatenated_datasets, hue="dataset_name", kind="scatter", plot_kws=dict(alpha=0.5),corner=True)

In [ ]:
sample_orginal=orginal_df.apply(lambda x: x.sample(frac=PLOT_FRACTION, random_state=2)) 
print(sample_orginal.shape)
print(sample_orginal.head())

In [ ]:
#sns.pairplot(sample_orginal, kind="scatter")

In [ ]:
sample_decoded=decoded_df.apply(lambda x: x.sample(frac=PLOT_FRACTION, random_state=2)) 
print(sample_decoded.shape)
print(sample_decoded.head())

In [ ]:
# sns.pairplot(sample_decoded, kind="scatter")

In [ ]:
# g = sns.PairGrid(concatenated_datasets, hue='dataset_name')
# g.map_upper(sns.scatterplot)
# #g.map_lower(sns.kdeplot, fill=True)
# g.map_diag(sns.histplot, kde=True)

In [ ]:
tmp_2=df_data.iloc[37:38,:]
tmp_2=tmp_2.to_numpy(dtype=np.float32)
orginal_2=tmp_2
tmp_2=stdcs.transform(tmp_2)
tmp_2=torch.from_numpy(tmp_2)
model.eval()
with torch.no_grad():
    result_encoded_features_2, z_mean_2, z_log_var_2, result_decoded_features_2=model(tmp_2.to(device=DEVICE))
result_2=result_decoded_features_2.cpu().detach().numpy()
result_2=stdcs.inverse_transform(result_2)

In [ ]:
orginal_2=orginal_2.flatten()
result_2=result_2.flatten()

In [ ]:
df=pd.DataFrame(np.stack((orginal_2,result_2)), columns=['X', 'Y', 'dX', 'dY', 'dZ', 'E'])
df

In [ ]:
encoded_features=[]
model.eval()
for index, feature in enumerate(train_loader):
    with torch.no_grad():
        tmp_encoded_features, z_mean, z_log_var, decoded =model(feature.to(device=DEVICE))
        encoded_features.extend(tmp_encoded_features.cpu().detach().numpy())
encoded_features=np.asarray(encoded_features)

print(encoded_features.shape)

In [ ]:
#PRZESTRZEŃ UKRYTA VAE
colors = ['r']
markers = ['s']

fig=plt.figure()
ax=fig.add_subplot(projection='3d')
for c, m in zip(colors, markers):
    ax.scatter(encoded_features[:,0],encoded_features[:,1],encoded_features[:,2],marker=m,c=c)

ax.set_xlabel('VAE 1')
ax.set_ylabel('VAE 2')
ax.set_zlabel('VAE 3')

plt.show()